In [0]:
from keras.models import Model
from keras.layers import BatchNormalization, Input, ReLU, Flatten, Dense
from keras.layers import Conv2D, AvgPool2D, DepthwiseConv2D
from keras.engine.topology import get_source_inputs
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.datasets import cifar100
import numpy as np

In [0]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
y_train = np_utils.to_categorical(y_train, 100)
y_test = np_utils.to_categorical(y_test, 100)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train  /= 255
x_test /= 255

In [0]:
def mobilenet(input_shape, n_classes):
  
  def conv_bn_rl(x, f, k=1, s=1, p='same'):
    x = Conv2D(f, k, strides=s, padding=p)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x
  
  
  def mobilenet_block(x, f, s=1):
    x = DepthwiseConv2D(3, s, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = conv_bn_rl(x, f)
    return x
  
  
  input = Input(input_shape)
  
  x = conv_bn_rl(input, 32, 3, 2)
  x = mobilenet_block(x, 64)
  
  x = mobilenet_block(x, 128, 2)
  x = mobilenet_block(x, 128)
  
  x = mobilenet_block(x, 256, 2)
  x = mobilenet_block(x, 256)
  
  x = mobilenet_block(x, 512, 2)
  for i in range(5):
    x = mobilenet_block(x, 512)
  
  x = mobilenet_block(x, 1024, 2)
  x = mobilenet_block(x, 1024)
  
  x = AvgPool2D(7)(x)
  
  x = Flatten()(x)
  output = Dense(n_classes, activation='softmax')(x)
  
  model = Model(input, output)
  return model

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    x_train,
    target_size=(299, 299),
    batch_size=64
    class_mode='binary'
)

In [3]:
input_shape = 299, 299, 3
n_classes = 100

model = mobilenet(input_shape, n_classes)
model.summary()
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 299, 299, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 32)      896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 150, 150, 32)      128       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 150, 150, 32)      0         
_________________________________________________________________
depthwise_conv2d_1 (Depthwis (None, 150, 150, 32)      320       
_________________________________________________________________
batch_normalization_2 (Batch (None, 150, 150, 32)      128       
_________________________________________________________________
re_lu_2 (ReLU)               (None, 150, 150, 32)      0         
__________

In [0]:
|datagen = ImageDataGenerator(horizontal_flip=True, width_shift_range=0.1, height_shift_range=0.1)

datagen.fit(x_train)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=64),
                    steps_per_epoch=x_train.shape[0] // 64,
                    epochs=60,
                    verbose=1,
                    validation_data=(x_test, y_test))

model.evaluate(x_test, y_test)